In [3]:
from tdw.controller import Controller
from tdw.tdw_utils import TDWUtils
from tdw.add_ons.third_person_camera import ThirdPersonCamera
from typing import Dict
from tdw.add_ons.collision_manager import CollisionManager
from tdw.add_ons.object_manager import ObjectManager

# Added for video
from tdw.add_ons.image_capture import ImageCapture
from tdw.backend.paths import EXAMPLE_CONTROLLER_OUTPUT_PATH
from os import chdir, system
from subprocess import call
import shutil

# Added for collisions
import random
import numpy as np

# From &tdw_physics
from typing import List, Dict
from tdw.librarian import ModelLibrarian
from typing import Dict
from helpers import ObjectInfo, get_random_avatar_position

In [11]:
lib = ModelLibrarian('models_core.json')
records = {record.name:i for i, record in enumerate(lib.records)}

In [31]:
lib = ModelLibrarian('models_full.json')
lib.records[0].__dict__

{'name': '004_rose_pink',
 'urls': {'Darwin': 'https://tdw-private.s3.amazonaws.com/models/osx/2018-2019.1/004_rose_pink',
  'Linux': 'https://tdw-private.s3.amazonaws.com/models/linux/2018-2019.1/004_rose_pink',
  'Windows': 'https://tdw-private.s3.amazonaws.com/models/windows/2018-2019.1/004_rose_pink'},
 'wnid': 'n11669921',
 'wcategory': 'flower',
 'scale_factor': 1.0,
 'do_not_use': False,
 'do_not_use_reason': '',
 'flex': True,
 'substructure': [{'materials': ['003-004_rose_bush_pink'],
   'name': '004_rose_pink'}],
 'bounds': {'back': {'x': -5.960464e-08, 'y': 0.5705476, 'z': -0.3926587},
  'bottom': {'x': -5.960464e-08, 'y': 0, 'z': -2.980232e-08},
  'center': {'x': -5.960464e-08, 'y': 0.5705476, 'z': -2.980232e-08},
  'front': {'x': -5.960464e-08, 'y': 0.5705476, 'z': 0.3926587},
  'left': {'x': -0.4108386, 'y': 0.5705476, 'z': -2.980232e-08},
  'right': {'x': 0.4108385, 'y': 0.5705476, 'z': -2.980232e-08},
  'top': {'x': -5.960464e-08, 'y': 1.141095, 'z': -2.980232e-08}},
 '

In [50]:
len(lib.records[2049].__dict__.keys())
len(lib.records[0].__dict__.keys())


17

In [ ]:
lib.records

In [52]:
lib = ModelLibrarian('models_flex.json')
lib.records[0].__dict__
for record in lib.records:
    if record.flex == True:
        print(record.name)
        

bowl
cone
cube
cylinder
dumbbell
octahedron
pentagon
pipe
platonic
pyramid
sphere
torus
triangular_prism


: 

In [21]:
lib.records[0].name

'034_vray'

In [22]:
print(records['rh10'])

445


In [23]:
records = {record.name:record for record in lib.records}

In [24]:
records['rh10'].name

'rh10'

In [ ]:
x = 5

In [ ]:
x = 6 if True == False

In [ ]:
print(x)